In [ ]:
# importing all necessary libraries
import numpy as np
import random as rd
import time
import numpy as np
from matplotlib import pyplot as plt
# from matplotlib import widgets
import matplotlib
from matplotlib import cm as cm
import scipy as SP
from PIL import Image
import threading
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from scipy import stats
from IPython.display import clear_output
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
matplotlib.use('TkAgg')

### Environment Class

The `Environment` class has been used to initialize the grid-based environment in which the package delivery agent operates. It is initialized with a grid size, random agent starting location and package location 'A'. The environment has a fixed location for destination 'B', which is the bottom-right corner of the grid.

**Methods:**

- `set_agent_start(agent)`: Sets the agent's starting location in the environment.
- `move_agent(agent_move)`: Updates the agent's location on the grid in respect to the movement action taken by the agent.
- `get_grid()`: This will give the current state of the grid.
- `plot_grid(snapshot, ax=None)`: This is the implementation of the visualization for the grid state throughout the journey of the agent from its start location to end destination. Visualization is implemented using Matplotlib. It displays images representing the agent, package, and destination locations.

### Reward Structure Explanation:

In the context of the package delivery problem, a reward structure is used to provide feedback to the agent's decisions and actions. This feedback is used as a guide to urge the agent towards making optimal decisions to achieve the goal of delivering the package efficiently.

- `reward_pickup`: A positive reward given to the agent when it picks up the package. It encourages the agent to collect the package.
- `reward_deliver`: A positive reward given to the agent when it successfully delivers the package to the destination. It incentivizes the agent to complete the delivery.
- `reward_move`: A negative reward given to the agent for each move it makes. If the move is not helping it reach the package or deliver it, the reward will be negative. This will ensure that the agent minimizes unnecessary movements.

These reward values are used to help the agent learn via reinforcement learning, over the period of training the agent will learn the best actions and decisions to make that will maximize its cumulative reward over time. It will learn to pick up the package, deliver it to the destination, and minimize the number of moves required to successfully complete this task.

In [ ]:
class Environment():
    def __init__(self, size:int, a = None):
        """
          Initialize a grid of size defined by parameter
          b; a tuple repsenting the delivery location
          a; a tuple representing the package location
          size; an int representing the horizontal and vertical dimensions of the grid
        """
        self.size = size
        self.b = (size-1, size-1)
        self.grid, self.a = self.setup_grid(size, a)
        self.reward_pickup = 100
        self.reward_deliver = 100
        self.reward_move = -1

    #method to set up the original grid including a location
    def setup_grid(self, size, a):
        grid = np.zeros((size,size))
        if a is None:
            x, y = rd.randint(0, size-1), rd.randint(0, size-1)
            while (x,y) == self.b:
                x, y = rd.randint(0, size-1), rd.randint(0, size-1)
            grid[x,y] = 1  #A represented by 1
        else:
            x,y=a
            grid[x,y] = 1
        grid[size-1,size-1] = 2  #B represented by 2
        return grid, (x,y)

    #method to add start location of agent to grid as well
    def set_agent_start(self, agent):
        self.agent_coords = (agent.x, agent.y)

    #Method which updates the location of the agent on the grid. Currently just zeroes whatever it landed on - can include other logic instead
    def move_agent(self, agent_move):
        x, y = self.agent_coords
        self.grid[x,y] = 0
        self.agent_coords = agent_move[0] , agent_move[1]
        self.grid[agent_move[0] , agent_move[1]] = -1

    def get_grid(self):
        return self.grid.tolist()

    # visulaize the agent movement through iterations
    def plot_grid(self, snapshot, ax=None):
        if ax is None:
            fig, ax = plt.subplots()
            ax.set_facecolor('white')
        else:
            ax.clear()

        # Plot the grid
        ax.imshow(np.array([[0]]), cmap="bone", extent=[0, self.size, 0, self.size])

        for i in range(self.size):
            for j in range(self.size):
                cell_value = snapshot[i][j]
                if cell_value == -1:
                    # Display agent image in the cell
                    imagebox = OffsetImage(agent_img, zoom=0.08)
                    ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)
                elif cell_value == 1:
                    # Display package image in the cell
                    imagebox = OffsetImage(package_img, zoom=0.03)
                    ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)
                elif cell_value == 2:
                    # Display destination image in the cell
                    imagebox = OffsetImage(destinationB_img, zoom=0.05)
                    ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)
                else:
                    ax.text(j + 0.5, self.size - i - 0.5, self.grid[i, j], ha='center', va='center', fontsize=20, color='black')

        # Set axis properties
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_xticks(np.arange(self.size) + 1)
        ax.set_yticks(np.arange(self.size) + 1)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.grid(True, linewidth=2, color='white')

        # Set title
        ax.set_title("Package Delivery Agent")

        # Show the plot
        #plt.show()
        return ax

agent_img = plt.imread('agent.jpg')
package_img = plt.imread('package.jpg')
destinationB_img = plt.imread('destinationB.jpg')

### Agent Class
The `Agent` class is used to intialize the agent that interacts with the environment and learns to make decisions.

**Methods:**
- `choose_best_option(qmat)`: Uses the q-matrix and determines the best action based on the greedy policy. It accordingly returns which action should be taken to the agent.
- `move(direction)`: makes the agent move by updating the position based on the chosen action. The new state and associated reward is updated.
- `reset_agent()`: resets the agent's state to a random position within the environment.

### Hypers Class
The Hypers class is defined to store the hyperparameters used in reinforcement learning process. They will influence the agent's learning behavior & exploration strategy. The hyperparameters will determine how the agent explores the environment, learns from experiences, and makes updated to take informed decisions.

In [ ]:
class Agent():
    def __init__(self, environ, opts):
        self.environ = environ
        self.reset_agent()
        self.opts = opts

    # choose the best action based on the greedy policy on Q-values
    # filed qmat = Q-matrix of size n*n*options where n=grid_size, options=actions
    def choose_best_option(self, qmat):
        if self.collected == 0:
            state_qs = qmat[self.x-self.environ.a[0] + self.environ.size-1,
                            self.y-self.environ.a[1] + self.environ.size-1,
                            self.collected] # 1*1*options
        else:
            state_qs = qmat[self.x-self.environ.b[0] + self.environ.size-1,
                            self.y-self.environ.b[1] + self.environ.size-1,
                            self.collected] # 1*1*options
        action = np.argmax(state_qs) # choose action based on greedy policy where 0=up, 1=down, 2=left, 3=right
        return action

    # allow agent to move by taking action up, down, left and right.
    def move(self, direction):

        if direction == self.opts[0]:
            if self.x-1 >= 0:
                self.x = self.x-1
        if direction == self.opts[1]:
            if self.x+1 < self.environ.size:
                self.x = self.x+1
        if direction == self.opts[2]:
            if self.y-1 >= 0:
                self.y = self.y-1
        if direction == self.opts[3]:
            if self.y+1 < self.environ.size:
                self.y=self.y+1

        if self.collected == 0:
            if (self.x, self.y) == self.environ.a:
                self.collected = 1
                reward = self.environ.reward_pickup
            else:
                reward = self.environ.reward_move

        else:
            if (self.x, self.y) == self.environ.b:
                self.end_episode = 1
                reward = self.environ.reward_deliver
            else:
                reward = self.environ.reward_move

        return (self.x, self.y, self.collected), reward, self.end_episode

    def reset_agent(self):
        self.x, self.y = rd.randint(0, self.environ.size-1), rd.randint(0, self.environ.size-1)
        while ((self.x, self.y) == (self.environ.a or self.environ.b)):
            self.x, self.y = rd.randint(0, self.environ.size-1), rd.randint(0, self.environ.size-1)
        self.collected = 0
        self.end_episode = 0

#Class to store hyperparameters
class Hypers():
    def __init__(self, eps, gamma, alpha, steps_per_ep, total_iter):
        """
            Initializes the class with the following hyperparameters:
          eps: Exploration factor (epsilon) defines the exploration - exploitation probability.
          gamma: Discount factor that determines the importance of future rewards in the agent's decision-making.
          alpha: Learning rate controls how quickly the agent adapts to new information to learn it.
          steps_per_ep: The maximum number of steps allowed per episode during training.
          total_iter: The total number of training iterations or episodes that the agent will undergo.
        """
        self.eps = eps
        self.gamma = gamma
        self.alpha = alpha
        self.steps_per_ep = steps_per_ep
        self.total_iter = total_iter

###Q-Learning class:
The QL class trains the agent usiing Q-Leaarning algorithm.

**Attributes:**

- `env_size (int)`: Size of the environment grid.

- `agent_opts (list)`: List of available agent movement options.

- `hypers (object)`: Hyperparameters for training.

- `qmat (numpy.ndarray)`: Q-value matrix to store Q-values for state-action pairs.
- `distance_history (list)`: List to store the difference between chosen and optimal paths during training.

**Methods:**

- `__init__(self, env_size, agent_opts, hypers)`:Constructor for QL class.
        
- `train(self, save_every=500)`:Train the agent using Q-learning algorithm.
        
- `load_qmat(self, filename, overwrite=False)`: Load a Q-value matrix from a file.
        
- `save_qmat(self, name)`:Save the current Q-value matrix to a file.



In [ ]:
class QL:
    def __init__(self, env_size, agent_opts, hypers):
       """
        Initialize QL class with the following arguments.
            env_size (int): Size of the environment grid.
            agent_opts (list): List of available agent movement options.
            hypers (object): Hyperparameters for training.
        """
        self.env_size = env_size
        self.agent_opts = agent_opts
        self.hypers = hypers
        self.qmat = np.zeros([2*(env_size-1)+1, 2*(env_size-1)+1, 2, len(agent_opts)])
        self.distance_history = []

    def train(self, save_every = 500):
      """
        Train the agent using Q-learning algorithm.

        Args:
            save_every (int): Frequency of saving the Q-value matrix.

        Returns:
            numpy.ndarray: Trained Q-value matrix.
        """
        for i in range(self.hypers.total_iter):
            # Create a new environment and agent for each iteration
            env = Environment(size=self.env_size)
            agent = Agent(env, self.agent_opts)
            env.set_agent_start(agent)
            agent.reset_agent()

            #calculate length of optimal path
            optimal_path = abs(agent.x-env.a[0]) + abs(agent.y-env.a[1]) + abs(env.a[0]-env.b[0]) + abs(env.a[1]-env.b[1])

            for s in range(self.hypers.steps_per_ep):
                # Choose between explore and exploit (epsilon-greedy)
                if rd.random() < (self.hypers.eps)**(i+1): #epsilon-decay for each episode
                    action = rd.randint(0, len(self.agent_opts)-1)
                else:
                    action = agent.choose_best_option(self.qmat)

                if agent.collected == 0:
                    state = (agent.x-env.a[0] + self.env_size-1, agent.y-env.a[1] + self.env_size-1, agent.collected)
                else:
                    state = (agent.x-env.b[0] + self.env_size-1, agent.y-env.b[1] + self.env_size-1, agent.collected)

                # Perform agent's action and receive new position, reward, and end of episode flag
                new_agent_pos, reward, end_episode = agent.move(self.agent_opts[action])
                if agent.collected == 0:
                    next_state = (new_agent_pos[0]-env.a[0] + self.env_size-1, new_agent_pos[1]-env.a[1] + self.env_size-1, agent.collected)
                else:
                    next_state = (new_agent_pos[0]-env.b[0] + self.env_size-1, new_agent_pos[1]-env.b[1] + self.env_size-1, agent.collected)

                # Determine the best action in the next state
                next_action = np.argmax(self.qmat[next_state])

                # Update Q-values using Q-learning equation
                self.qmat[state][action] += self.hypers.alpha * (reward + (self.hypers.gamma * (self.qmat[next_state][next_action])) - self.qmat[state][action])

                if end_episode == 1:
                    break

            #Add diff between chosen and optimal paths to training history
            self.distance_history.append(s - optimal_path + 1)

            #save q_matrix to logs for future operation
            if i%save_every == 0:
                self.save_qmat(f"logs/episode{i}training.npy")
        return self.qmat

    def load_qmat(self, filename, overwrite = False):
      """
        Load a Q-value matrix from a file.

        Args:
            filename (str): Name of the file to load the Q-value matrix from.
            overwrite (bool): If True, overwrite the current Q-value matrix with the loaded one.

        Returns:
            numpy.ndarray: Loaded Q-value matrix.
        """
        qmat = np.load(filename, allow_pickle=False)
        if overwrite:
            self.qmat = qmat
        return qmat

    def save_qmat(self, name):
      """
        Save the current Q-value matrix to a file.

        Args:
            name (str): Name of the file to save the Q-value matrix to.
        """
        np.save(name, self.qmat, allow_pickle=False)



###Train QL agent:
This code demonstrates training of a Q-Learning agent in the grid environment.
1. Set the environment size (env_size) and available movement options (opts).
2. Define hyperparameters using the Hypers class to configure the training process.
3. Initialize a QL (Q-Learning) agent with the specified environment size, movement options, and hyperparameters.
4. Train the agent using the train() method to obtain the Q-value matrix.

In [ ]:
# Train
env_size = 5  # Set your environment size
opts = ["up", "down", "left", "right"]
# Configure Hyperparameters for Training
hypers = Hypers(
    eps=0.4,            # Epsilon value for epsilon-greedy exploration
    gamma=0.8,          # Discount factor for future rewards in Q-learning
    alpha=0.01,         # Learning rate for updating Q-values
    steps_per_ep=100,   # Maximum number of steps per episode
    total_iter=20000    # Total number of training iterations
    # eps_decay_rate=1000
)
# Initialize QL Agent and Train
"""
Create an instance of the QL class with the specified environment size, movement options, and hyperparameters.
Train the agent using the train() method to learn Q-values and obtain the Q-value matrix (qmat).
"""
ql = QL(env_size, opts, hypers)
qmat = ql.train()  # Train the agent and obtain the Q-value matrix

###Run visualization:
 Runs a visualization of the agent's movement in the environment.

**Attributes:**
- `env (Environment)`: The environment instance where the agent is placed.
- `agent (Agent)`: The agent instance to be visualized.
- `qmat (numpy.ndarray)`: The Q-value matrix used for making decisions.


Run visualization function simulates the agent's movement in the environment based on the actions selected using the Q-value matrix.The function terminates either after a fixed number of steps or when the agent reaches an end state (as indicated by end == 1).

In [ ]:
env = Environment(size=5)
agent = Agent(env, opts)

def run_visualisation(env, agent, qmat):
  # Set the agent's starting position in the environment
    env.set_agent_start(agent)
    env.move_agent((agent.x, agent.y, agent.collected))
    snapshots=[]

    # Create a figure and axis outside the loop
    fig, ax = plt.subplots()
    ax.set_facecolor('white')

    # Initialize the plot once with the initial grid
    im = ax.imshow(env.grid, cmap="bone", extent=[0, env.size, 0, env.size])

    # Run the simulation steps
    for step in range(40):
        action = agent.choose_best_option(qmat)# Choose the best action according to the Q-value matrix
        pos, _, end = agent.move(agent.opts[action])
        env.move_agent(pos)
        snapshots.append(env.get_grid())
        env.plot_grid(snapshots[step], ax)

        # Redraw the plot
        fig.canvas.draw()
        plt.pause(0.2)  # Add a short pause for visualization
        if end == 1:
            break

run_visualisation(env, agent, qmat)

In [ ]:
#Generate and display a graph of the training history.
def graph_training_history(ql):
    plt.plot(ql.distance_history)  # Plot the distance history data
    plt.xlabel("Episode")          # Set the label for the x-axis
    plt.ylabel("Distance From Optimal Solution")  # Set the label for the y-axis
    plt.title("Training History")   # Set the title for the graph
    plt.show()                     # Display the generated graph

graph_training_history(ql)# Call the function to generate and display the graph